In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import re
from fractions import Fraction
import seaborn as sns

In [2]:
def split_damage_rolls(damage_string):
    '''
    Given a damage_string, determine if more than one are present and if so, split them. 
    e.g., input --> 1d4 returns 1d4, but input --> 1d4+2d6 returns ['1d4',2d6']. 
    Function also catches modifiers, e.g., input--> 6d12+5+1d6-2 returns ['6d12+5','1d6-2']
    '''
    d_locs = [m.start() for m in re.finditer('d', damage_string)]
    d2 = d_locs[-1]
    plus_locs= [m.start() for m in re.finditer('\+', damage_string)]
    diffs = d2 - np.array(plus_locs)
    corr_ind = d2 - np.min([n for n in diffs  if n>0])
    return [damage_string[:corr_ind],damage_string[corr_ind+1:]]
    
def calc_max_damage(damage_string):
    '''
    Given a string in the d&d format 1d8+5 (etc.) or supplemental format 1d8+4+1d6+2,
    return the maximum damage for that roll.
    '''
    if damage_string == 'nan' or damage_string=='VARIES':
        return 0.0
    else:
        mult = damage_string.replace('d','*')
        max_dmg = eval(mult)
        return max_dmg
    
    
def calc_avg_damage(damage_string,combine=True):
    '''
    Given a string in the d&d format 1d8+5 (etc.) or supplemental format 1d8+4+1d6+2,
    return the average damage for that roll.
    '''
    if damage_string == 'nan' or damage_string=='VARIES' or damage_string == 'NaN':
        return 0.0
    else:
        num_roll = damage_string.count('d')
        if num_roll > 1:
            rolls = split_damage_rolls(damage_string)
        else:
            rolls = [damage_string]

        avg_damages = []
        for dmg_str in rolls:
            pos_d = dmg_str.find('d')
            if pos_d == -1:
                return 0.0
            num_dice = int(dmg_str[:pos_d])
            mult = dmg_str.replace('d','*0.5*')
            avg_dmg = eval(mult) 
            avg_damages.append(avg_dmg + 0.5*num_dice)

        if combine:
            return np.sum(avg_damages)
        elif combine==False:
            return avg_damages

In [3]:
def statblock_to_structure(df_crc1):
    '''
    Turn full statblock into structure matrix precursor.
    Takes in statblock dataframe
    This function drops name and CR, renormalizes columns, and generates column combinations. 
    Returns structure matrix dataframe
    '''
    df_crc1=df_crc1.drop(['Name','CR'],axis=1)
    for c in df_crc1.columns:
        df_crc1[c] = df_crc1[c]/df_crc1[c].mean()
    df_crc = df_crc1.copy()
    
    print(df_crc1.columns)
    for ci in df_crc1.columns:
        for cj in df_crc1.columns:
            if not str(ci+'*'+cj) in df_crc.columns:
                df_crc[str(ci+'*'+cj)]=df_crc1[ci]*df_crc1[cj]
            
    return df_crc

In [4]:
df = pd.read_csv('../csv/monster_stats_with_attack_rolls.csv',header=0,na_values='NaN')
df2 = pd.read_csv('../csv/monster_stats.csv',header=0,na_values='NaN')

def load_statblocks(df,df2):
    df = df.drop(['alignment','Book','Desert','Forest','Grassland','Hill','Mountain','Swamp','Underdark','Underwater','Urban','Arctic','Coast','CR_decimal','page'],axis=1)
    df = df.drop(df.index[555:])
    df['attack_1_dmg'] = df['attack_1_dmg'].astype(str)
    df['attack_2_dmg'] = df['attack_2_dmg'].astype(str)
    df['CR'] = [float(Fraction(s)) for s in df.CR]
    df['attack_1_max_dmg'] = [calc_max_damage(i) for i in df['attack_1_dmg'].values]
    df['attack_2_max_dmg'] = [calc_max_damage(i) for i in df['attack_2_dmg'].values]
    df['attack_1_avg_dmg'] = [calc_avg_damage(i) for i in df['attack_1_dmg'].values]
    df['attack_2_avg_dmg'] = [calc_avg_damage(i) for i in df['attack_2_dmg'].values]
    df['max_attack_dmg'] = df.attack_1_max_dmg + df.attack_2_max_dmg
    df['avg_attack_dmg'] = df.attack_1_avg_dmg + df.attack_2_avg_dmg
    df['spellcasting'] = np.nan
    df.loc[df['spellcasting_ability']=='YES','spellcasting'] = 1.0
    df.loc[df['spellcasting_ability']=='NO','spellcasting'] = 0.0
    df = df.drop(['spellcasting_ability'],axis=1)
    #df = df.set_index('name')
    
    dft=df2.merge(df,on='Name',how='inner')

    dfu = pd.DataFrame(dft.Name)
    dfu['CR']=dft.CR_x.astype('float')
    dfu['HP']=dft.HP_x.astype('float')
    dfu['AC']=dft.AC_x.astype('float')
    dfu['STR']=dft.STR.astype('float')
    dfu['DEX']=dft.DEX.astype('float')
    dfu['CON']=dft.CON.astype('float')
    dfu['INT']=dft.INT.astype('float')
    dfu['WIS']=dft.WIS.astype('float')
    dfu['CHA']=dft.CHA.astype('float')
    dfu['avg_attack_1']=dft.attack_1_avg_dmg
    dfu['avg_attack_2']=dft.attack_2_avg_dmg
    dfu['max_attack_1']=dft.attack_1_max_dmg
    dfu['max_attack_2']=dft.attack_2_max_dmg
    dfu['spellcasting']=dft.spellcasting
    nsize = dft.Size
    nsize[nsize=='Tiny']=0
    nsize[nsize=='tiny']=0
    nsize[nsize=='Small']=1
    nsize[nsize=='small']=1
    nsize[nsize=='Medium']=2
    nsize[nsize=='medium']=2
    nsize[nsize=='Large']=3
    nsize[nsize=='large']=3
    nsize[nsize=='Huge']=4
    nsize[nsize=='huge']=4
    nsize[nsize=='Gargantuan']=5
    nsize[nsize=='gargantuan']=5
    dfu['Size']=nsize

    
    return dfu
    

In [ ]:
dfu_test = load_statblocks(df,df2)

In [ ]:
statblock_to_structure(dfu_test)